<a href="https://colab.research.google.com/github/shahzaibkhanniazi-dot/MSAI-VetDisease-CRNN-2026/blob/main/MSAI_VetDisease_CRNN_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Initial Setup & Mounting

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
import random
import numpy as np

# Seeding for accuracy
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Choosing GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Mounted at /content/drive
Using device: cuda


2. Loading the Veterinary Data

In [2]:
# Preparing the images
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Linking to the data folder
data_dir = '/content/drive/MyDrive/MSAI_Project_2026/data'

# Splitting data for training and testing
dataset = datasets.ImageFolder(data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Setting up the data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Disease Classes: {dataset.classes}")

Disease Classes: ['Bacterial_dermatosis', 'Fungal_infections', 'Healthy', 'Hypersensitivity_allergic_dermatosis']


3. Building the Model Brain (CRNN)

In [3]:
# Creating the CRNN brain from scratch
class VeterinaryCRNN(nn.Module):
    def __init__(self, num_classes=4):
        super(VeterinaryCRNN, self).__init__()

        # Extracting image features
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # Understanding patterns with RNN
        self.rnn = nn.GRU(input_size=64 * 32, hidden_size=128, num_layers=2, batch_first=True)

        # Final decision layer
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.cnn(x)
        # Reshaping for the RNN
        x = x.permute(0, 2, 3, 1).contiguous()
        x = x.view(x.size(0), x.size(1), -1)

        x, _ = self.rnn(x)
        x = x[:, -1, :] # Getting the final result
        x = self.fc(x)
        return x

# Initializing our custom model
model = VeterinaryCRNN(num_classes=len(dataset.classes)).to(device)
print(model)

VeterinaryCRNN(
  (cnn): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (rnn): GRU(2048, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=4, bias=True)
)
